# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 5 2022 9:05AM,241043,10,8208600,Eywa Pharma Inc.,Released
1,Jul 5 2022 9:05AM,241043,10,8218241,Eywa Pharma Inc.,Released
2,Jul 5 2022 9:03AM,241042,19,TR-RA-06232022,"GCH Granules USA, Inc.",Released
3,Jul 5 2022 8:57AM,241040,10,0085828493,ISDIN Corporation,Released
4,Jul 5 2022 8:57AM,241040,10,0085828494,ISDIN Corporation,Released
5,Jul 5 2022 8:57AM,241040,10,0085828506,ISDIN Corporation,Released
6,Jul 5 2022 8:57AM,241040,10,0085828499,ISDIN Corporation,Released
7,Jul 5 2022 8:57AM,241040,10,0085828504,ISDIN Corporation,Released
8,Jul 5 2022 8:57AM,241040,10,0085828508,ISDIN Corporation,Released
9,Jul 5 2022 8:57AM,241040,10,0085828507,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,241038,Released,50
37,241039,Released,62
38,241040,Released,60
39,241042,Released,1
40,241043,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241038,NaN,50.0
241039,NaN,62.0
241040,NaN,60.0
241042,NaN,1.0
241043,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241002,0.0,55.0
241003,8.0,40.0
241004,0.0,59.0
241005,0.0,54.0
241006,0.0,55.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241002,0,55
241003,8,40
241004,0,59
241005,0,54
241006,0,55


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241002,0,55
1,241003,8,40
2,241004,0,59
3,241005,0,54
4,241006,0,55


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241002,,55
1,241003,8,40
2,241004,,59
3,241005,,54
4,241006,,55


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 5 2022 9:05AM,241043,10,Eywa Pharma Inc.
2,Jul 5 2022 9:03AM,241042,19,"GCH Granules USA, Inc."
3,Jul 5 2022 8:57AM,241040,10,ISDIN Corporation
63,Jul 5 2022 8:57AM,241039,10,ISDIN Corporation
125,Jul 5 2022 8:57AM,241038,10,ISDIN Corporation
175,Jul 5 2022 8:57AM,241035,10,ISDIN Corporation
229,Jul 5 2022 8:45AM,241037,10,"Amcyte Pharma, Inc."
233,Jul 5 2022 8:43AM,241034,10,Bio-PRF
246,Jul 5 2022 8:43AM,241036,10,MedStone Pharma LLC
247,Jul 5 2022 8:35AM,241033,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 5 2022 9:05AM,241043,10,Eywa Pharma Inc.,,2
1,Jul 5 2022 9:03AM,241042,19,"GCH Granules USA, Inc.",,1
2,Jul 5 2022 8:57AM,241040,10,ISDIN Corporation,,60
3,Jul 5 2022 8:57AM,241039,10,ISDIN Corporation,,62
4,Jul 5 2022 8:57AM,241038,10,ISDIN Corporation,,50
5,Jul 5 2022 8:57AM,241035,10,ISDIN Corporation,,54
6,Jul 5 2022 8:45AM,241037,10,"Amcyte Pharma, Inc.",,4
7,Jul 5 2022 8:43AM,241034,10,Bio-PRF,,13
8,Jul 5 2022 8:43AM,241036,10,MedStone Pharma LLC,,1
9,Jul 5 2022 8:35AM,241033,10,ISDIN Corporation,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 9:05AM,241043,10,Eywa Pharma Inc.,2,
1,Jul 5 2022 9:03AM,241042,19,"GCH Granules USA, Inc.",1,
2,Jul 5 2022 8:57AM,241040,10,ISDIN Corporation,60,
3,Jul 5 2022 8:57AM,241039,10,ISDIN Corporation,62,
4,Jul 5 2022 8:57AM,241038,10,ISDIN Corporation,50,
5,Jul 5 2022 8:57AM,241035,10,ISDIN Corporation,54,
6,Jul 5 2022 8:45AM,241037,10,"Amcyte Pharma, Inc.",4,
7,Jul 5 2022 8:43AM,241034,10,Bio-PRF,13,
8,Jul 5 2022 8:43AM,241036,10,MedStone Pharma LLC,1,
9,Jul 5 2022 8:35AM,241033,10,ISDIN Corporation,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 9:05AM,241043,10,Eywa Pharma Inc.,2,
1,Jul 5 2022 9:03AM,241042,19,"GCH Granules USA, Inc.",1,
2,Jul 5 2022 8:57AM,241040,10,ISDIN Corporation,60,
3,Jul 5 2022 8:57AM,241039,10,ISDIN Corporation,62,
4,Jul 5 2022 8:57AM,241038,10,ISDIN Corporation,50,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 9:05AM,241043,10,Eywa Pharma Inc.,2,NaN
1,Jul 5 2022 9:03AM,241042,19,"GCH Granules USA, Inc.",1,NaN
2,Jul 5 2022 8:57AM,241040,10,ISDIN Corporation,60,NaN
3,Jul 5 2022 8:57AM,241039,10,ISDIN Corporation,62,NaN
4,Jul 5 2022 8:57AM,241038,10,ISDIN Corporation,50,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 9:05AM,241043,10,Eywa Pharma Inc.,2,0.0
1,Jul 5 2022 9:03AM,241042,19,"GCH Granules USA, Inc.",1,0.0
2,Jul 5 2022 8:57AM,241040,10,ISDIN Corporation,60,0.0
3,Jul 5 2022 8:57AM,241039,10,ISDIN Corporation,62,0.0
4,Jul 5 2022 8:57AM,241038,10,ISDIN Corporation,50,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,8676772,1517,17.0
19,241042,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,8676772,1517,17.0
1,19,241042,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,1517,17.0
1,19,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1517.0
1,19,Released,1.0
2,10,Executing,17.0
3,19,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19
Status,,
Executing,17.0,0.0
Released,1517.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19
0,Executing,17.0,0.0
1,Released,1517.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19
0,Executing,17.0,0.0
1,Released,1517.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()